In [1]:
manifest = {
    'memory': 1024,
    'requirements': [
        'pandas',
        'influxdb',
        'scikit_learn',
        'scipy',
        'afs',
        'boto==2.48.0'
    ]
}

In [2]:
!pip install boto==2.48.0

In [3]:
import pandas as pd
from influxdb import DataFrameClient
import datetime
import os
import boto
import boto.s3.connection
from boto.s3.key import Key

In [4]:
## InfluxDB Configuration
IDB_HOST = '192.168.123.245'
IDB_PORT = 8086
IDB_DBNAME = 'c9377a95-82f3-4af3-ac14-40d14f6d2abe'
#IDB_CHANNEL = '1Y520210100'

IDB_USER = 'c6e23c03-dd57-4c8f-a6e2-b683ad76e8e4'
IDB_PASSWORD = 'NSr8dUZ6meiRlal8zqGcV6avK'

KEYWORD = ''

## S3 Configuration
S3_HOST = '192.168.123.242'
S3_PORT = 8080
S3_ACCESS_KEY = 'ee7da6e1bba944829884747df01a1eee'
S3_SECRET_KEY = 'd737995ebd3e459db321fb119f006456'

S3_BUCKET_FROM = 'FOMOS-Y5'
S3_BUCKET_TO = 'FOMOS-Y5-SUMMARY'

NUM_SMB = 'smartbox22 Signal Data'
FILE_DEST = 'Raw Data-1-'
SAMPLE_RATE = 8192


In [5]:
def read_influxdb_data(host='192.168.123.245', 
                       port=8086, 
                       dbname = 'c9377a95-82f3-4af3-ac14-40d14f6d2abe', 
                       ChannelName='1Y520210100', 
                       time_start='', 
                       time_end='', 
                       user = 'c6e23c03-dd57-4c8f-a6e2-b683ad76e8e4', 
                       password = 'NSr8dUZ6meiRlal8zqGcV6avK',
                       keyword=''):
    
    #Example: read_influxdb_data(ChannelName='1Y520210200')
    #Example: read_influxdb_data(ChannelName='1Y520210200',time_start='2018-05-28',time_end='2018-05-29')
    client = DataFrameClient(host, port, user, password, dbname)
    measurements = client.get_list_measurements()
    
    if keyword is None: keyword = ''
        
    if keyword=='':
        measurement = [mea.get(u'name') for mea in measurements if mea.get(u'name').find(ChannelName)>=0]
    else:
        measurement = [mea.get(u'name') for mea in measurements if mea.get(u'name').find(ChannelName)>=0 and mea.get(u'name').find(keyword)>=0]
    
    if len(measurement)==0: 
        print('No data retrieved.')
        return None
    
    measurement = measurement[-1]
    
    time_end = 'now()' if time_end=='' else "'" + time_end + ' 15:59:00' + "'"
    print(time_end)
    
    time_start = 'now()' if time_start=='' else "'" + time_start + ' 16:00:00' + "'"
    print(time_start)
    
    querystr = 'select * from "{}" where time > {} and time < {}'.format(measurement,time_start,time_end)
    print(querystr)
    
    df = client.query(querystr).get(measurement)
    client.close()
    
    if df is None: 
        print('No data retrieved.')
        return None    
    
    dff = df.groupby('id')
    columns = [name for name, group in dff]
    groups = [group['val'] for name, group in dff]
    
    #check datatime alginment: all([all(groups[i].index==groups[0].index) for i in range(1,len(groups))])
    result = pd.concat(groups,axis=1)
    result.columns = columns
    result.index = groups[0].index
    
    print('data between {} and {} are retrieved, dimension: {}x{}'.format(time_start,time_end,result.shape[0],result.shape[1]))
    
    return measurement, result

# Ger Current Date

In [6]:
today = datetime.date.today() + datetime.timedelta(hours=8) - datetime.timedelta(days=1)

TIME_END = str(today.year) + '-'\
                + '{:02d}'.format(today.month) + '-'\
                + '{:02d}'.format(today.day)

pastday = datetime.date.today() + datetime.timedelta(hours=8) - datetime.timedelta(days=2)
TIME_START = str(pastday.year) + '-'\
                + '{:02d}'.format(pastday.month) + '-'\
                + '{:02d}'.format(pastday.day)

print ('from:', TIME_END, 'to', TIME_START)

from: 2018-06-28 to 2018-06-27


# Get data along with the date

In [7]:

def get_scada_data (ChannelName):
    measurement, data = read_influxdb_data(host = IDB_HOST,
                                       port = IDB_PORT,
                                       dbname = IDB_DBNAME,
                                       ChannelName = ChannelName,
                                       time_start = TIME_START,
                                       time_end = TIME_END,
                                       user = IDB_USER,
                                       password = IDB_PASSWORD
                                      )

    return data
#measurement, data = read_influxdb_data(ChannelName='1Y520210200')



In [8]:
#measurement, data = read_influxdb_data(host = IDB_HOST,
#                                       port = IDB_PORT,
#                                       dbname = IDB_DBNAME,
#                                       ChannelName = '1Y520220100',
#                                       time_start = TIME_START,
#                                       time_end = TIME_END,
#                                       user = IDB_USER,
#                                       password = IDB_PASSWORD
#                                      )

# Connect to S3 storege

In [9]:
s3_connection = boto.connect_s3(
                   aws_access_key_id = S3_ACCESS_KEY,
                   aws_secret_access_key = S3_SECRET_KEY,
                   host = S3_HOST,
                   port = S3_PORT,
                   is_secure=False,               # uncomment if you are not using ssl
                   calling_format = boto.s3.connection.OrdinaryCallingFormat(),
                 )

In [10]:
# From: 'FOMOS-Y5/smartbox11 signal Data/1Y510110100/2018/4/10/Raw Data-1-1Y510110100-00-05-05.bin'
# To: 'FOMOS-Y5-SUMMARY/smartbox11 signal Data/1Y510110100/2018/4/10/MAX-1Y510110100-1.bin'
# f = '2018-06-26 09:11:02'

def s3_cp (f='', type='', tag='', ChannelName=''):
    
    from_filename = FILE_DEST + ChannelName + '-' + \
                     str(f).split(' ')[1].split('+')[0].replace(':', '-') + '_' + str(SAMPLE_RATE) + '.bin'
    #print(from_filename)
    
    path = NUM_SMB + '/' + ChannelName + '/' \
                 + str(f).split('-')[0] + '/' + '{:01d}'.format(int(str(f).split('-')[1])) + '/' \
                 + '{:02d}'.format(int(str(f).split('-')[2].split(' ')[0]))
    
    to_filename = type + '-' +ChannelName + '-' + tag + '.bin'
    
    
    FROM = os.path.join(S3_BUCKET_FROM, path, from_filename)
    TO = os.path.join(S3_BUCKET_TO, path, to_filename)
    
    #print('From', FROM)
    #print('To', TO)
    
    # Download
    bucket = s3_connection.get_bucket(S3_BUCKET_FROM, validate=False)
    key = Key(bucket)
    key.key = os.path.join(path, from_filename)
    #print(key.key)
    key.get_contents_to_filename(from_filename)
    
    # Upload
    bucket = s3_connection.get_bucket(S3_BUCKET_TO, validate=False)
    key = Key(bucket)
    key.key = os.path.join(path, to_filename)
    key.set_contents_from_filename(from_filename)
    
    # Clean
    os.remove(from_filename)
    
    return 0

In [11]:
#print ('Raw Data-1-1Y520220101-03-30-38_8192.bin')
#s3_cp(f='2018-06-23 03:30:38', type='MEAN', tag=str(2))

# Sort features and figure out max, min and mean elements

In [12]:
def convert_taipei_time (datetime_object):

#datetime_object = '2018-06-26 23:16:32+00:00'
    datetime_object = datetime_object.split('+')[0]
    datetime_object = datetime.datetime.strptime(datetime_object, '%Y-%m-%d %H:%M:%S')
    datetime_object = datetime_object + datetime.timedelta(hours=8) 
    
    return datetime_object

In [13]:

def pre_process (Data='', ChannelName=''):
    
    for i in range(0, len(list(Data.columns.values))):

        #print(list(data.columns.values)[i])
        min = Data.sort_values(by=list(data.columns.values)[i]).index[0]
        mean = Data.sort_values(by=list(data.columns.values)[i]).index[len(data)//2]
        max = Data.sort_values(by=list(data.columns.values)[i]).index[-1]

        #print (min)
        min = convert_taipei_time(str(min))
        mean = convert_taipei_time(str(mean))
        max = convert_taipei_time(str(max))
        #print (min)

        s3_cp(f=min, type='MIN', tag=str(i), ChannelName=ChannelName)
        s3_cp(f=mean, type='MEAM', tag=str(i), ChannelName=ChannelName)
        s3_cp(f=max, type='MAX', tag=str(i), ChannelName=ChannelName)


In [14]:
bucket = s3_connection.get_bucket(S3_BUCKET_FROM, validate=False)

folders = bucket.list(NUM_SMB + "/","/")
for folder in folders:
    if folder.name.split("/")[1]:
        
        print (folder.name.split("/")[1])
        channel = folder.name.split("/")[1]
        data = get_scada_data (channel)
        pre_process (data, channel)

1Y520220100
'2018-06-28 15:59:00'
'2018-06-27 16:00:00'
select * from "HistRawData_4617a1aa-d153-4ae1-ae42-bff224af66d0_1Y520220100" where time > '2018-06-27 16:00:00' and time < '2018-06-28 15:59:00'
data between '2018-06-27 16:00:00' and '2018-06-28 15:59:00' are retrieved, dimension: 1416x47
1Y520220101
'2018-06-28 15:59:00'
'2018-06-27 16:00:00'
select * from "HistRawData_4617a1aa-d153-4ae1-ae42-bff224af66d0_1Y520220101" where time > '2018-06-27 16:00:00' and time < '2018-06-28 15:59:00'
data between '2018-06-27 16:00:00' and '2018-06-28 15:59:00' are retrieved, dimension: 1416x47
1Y520220102
'2018-06-28 15:59:00'
'2018-06-27 16:00:00'
select * from "HistRawData_4617a1aa-d153-4ae1-ae42-bff224af66d0_1Y520220102" where time > '2018-06-27 16:00:00' and time < '2018-06-28 15:59:00'
data between '2018-06-27 16:00:00' and '2018-06-28 15:59:00' are retrieved, dimension: 1416x47
1Y520220103
'2018-06-28 15:59:00'
'2018-06-27 16:00:00'
select * from "HistRawData_4617a1aa-d153-4ae1-ae42-bff22

KeyboardInterrupt: 